# logging

**ログレベルの説明**
|レベル|数値|説明|
|--|--|--|
|CRITICAL|50|致命的な状態でサービス提供ができないようなときのログに設定する。|
|ERROR|40|不具合が起きているときのログに設定する。|
|WARNING|30|エラーではが、潜在的に問題になる可能性があるときのログに設定する。|
|INFO|20|通常処理の中であとから確認する可能性がある情報などのログに設定する。ジョブの開始と終了の時刻など。|
|DEBUG|10|開発中に確認する必要のある情報を出力するときのログに設定する。|
  
**フォーマット文字列**
|表記|意味|
|--|--|
|%(asctime)s|日時|
|%(levelname)s|ログレベル|
|%(filename)s|ファイル名|
|%(module)s|モジュール名|
|%(lineno)d|行番号|
|%(funcName)s|関数、メソッド名|
|%(message)s|メッセージ|
|%(name)s|ロガー名|

## 一般的な従来の1件1行のログを出力する

In [1]:
from logging import (
    getLogger,
    StreamHandler,
    FileHandler,
    Formatter,
    DEBUG,
    INFO,
    WARNING,
    ERROR,
    CRITICAL
)

logger = getLogger(__name__)
# ロガーのログレベルを設定(ハンドラーよりも低い数値(DEBUGに近い)のログレベルにする必要がある)
logger.setLevel(DEBUG)

# フォーマットを設定
formatter = Formatter(
    "%(asctime)s - %(levelname)s : %(name)s : %(funcName)s : %(message)s (%(filename)s:%(lineno)d)"
)

# 標準出力用ハンドラーのログレベルを設定
s_handler = StreamHandler()
s_handler.setLevel(WARNING)
s_handler.setFormatter(formatter)
logger.addHandler(s_handler)

# ファイル出力用ハンドラーのログレベルを設定
f_handler = FileHandler(filename="./log/app.log", encoding="utf-8")
f_handler.setLevel(INFO)
f_handler.setFormatter(formatter)
logger.addHandler(f_handler)

# ログを出力
logger.critical("クリティカルのメッセージ")
logger.error("エラーのメッセージ")
logger.warning("ワーニングのメッセージ")
logger.info("インフォのメッセージ")
logger.debug("デバッグのメッセージ")

2024-12-30 00:02:53,326 - CRITICAL : __main__ : <module> : クリティカルのメッセージ (2987863191.py:35)
2024-12-30 00:02:53,328 - ERROR : __main__ : <module> : エラーのメッセージ (2987863191.py:36)
2024-12-30 00:02:53,329 - WARNING : __main__ : <module> : ワーニングのメッセージ (2987863191.py:37)


## JSON形式のログを出力する
※python-json-loggerのインストールが必要

In [2]:
!pip install python-json-logger


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from logging import (
    getLogger,
    StreamHandler,
    FileHandler,
    Formatter,
    DEBUG,
    INFO,
    WARNING,
    ERROR,
    CRITICAL,
)
from pythonjsonlogger import jsonlogger

logger = getLogger(__name__)
# ロガーのログレベルを設定(ハンドラーよりも低い数値(DEBUGに近い)のログレベルにする必要がある)
logger.setLevel(DEBUG)

# ※※JSONフォーマットを設定※※
formatter = jsonlogger.JsonFormatter(
    "%(asctime)s %(levelname)s %(name)s %(funcName)s %(message)s %(filename)s %(lineno)d",
    json_ensure_ascii=False,
)

# 標準出力用ハンドラーのログレベルを設定
s_handler = StreamHandler()
s_handler.setLevel(DEBUG)
s_handler.setFormatter(formatter)
logger.addHandler(s_handler)

# ファイル出力用ハンドラーのログレベルを設定
f_handler = FileHandler(filename="./log/app.log", encoding="utf-8")
f_handler.setLevel(DEBUG)
f_handler.setFormatter(formatter)
logger.addHandler(f_handler)

# ログを出力
logger.critical({"user_name": "tanaka", "message": "クリティカルのメッセージ"})
logger.error({"user_name": "tanaka", "message": "エラーのメッセージ"})
logger.warning({"user_name": "tanaka", "message": "ワーニングのメッセージ"})
logger.info({"user_name": "tanaka", "message": "インフォのメッセージ"})
logger.debug({"user_name": "tanaka", "message": "デバッグのメッセージ"})

{"asctime": "2024-12-30 00:08:50,814", "levelname": "CRITICAL", "name": "__main__", "funcName": "<module>", "message": "クリティカルのメッセージ", "filename": "782796838.py", "lineno": 37, "user_name": "tanaka"}
{"asctime": "2024-12-30 00:08:50,816", "levelname": "ERROR", "name": "__main__", "funcName": "<module>", "message": "エラーのメッセージ", "filename": "782796838.py", "lineno": 38, "user_name": "tanaka"}
{"asctime": "2024-12-30 00:08:50,817", "levelname": "WARNING", "name": "__main__", "funcName": "<module>", "message": "ワーニングのメッセージ", "filename": "782796838.py", "lineno": 39, "user_name": "tanaka"}
{"asctime": "2024-12-30 00:08:50,819", "levelname": "INFO", "name": "__main__", "funcName": "<module>", "message": "インフォのメッセージ", "filename": "782796838.py", "lineno": 40, "user_name": "tanaka"}
{"asctime": "2024-12-30 00:08:50,820", "levelname": "DEBUG", "name": "__main__", "funcName": "<module>", "message": "デバッグのメッセージ", "filename": "782796838.py", "lineno": 41, "user_name": "tanaka"}


## ログの設定をyamlから取得する
※pyyamlのインストールが必要

In [4]:
!pip install pyyaml


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 一般的な従来の1件1行のログを出力する

In [ ]:
from logging import getLogger, config
import yaml

# ログの設定を読み込み
config.dictConfig(
    yaml.load(
        open("./config/logging_nomal_format.yaml").read(),
        Loader=yaml.SafeLoader,
    )
)

# ロガーを取得
logger = getLogger(__name__)

# ログを出力
logger.critical("クリティカルのメッセージ")
logger.error("エラーのメッセージ")
logger.warning("ワーニングのメッセージ")
logger.info("インフォのメッセージ")
logger.debug("デバッグのメッセージ")

### JSON形式のログを出力する

In [1]:
from logging import getLogger, config
import yaml

# ログの設定を読み込み
config.dictConfig(
    yaml.load(
        open("./config/logging_json_format.yaml").read(),
        Loader=yaml.SafeLoader,
    )
)

# ロガーを取得
logger = getLogger(__name__)

# ログを出力
logger.critical({"user_name": "tanaka", "message": "クリティカルのメッセージ"})
logger.error({"user_name": "tanaka", "message": "エラーのメッセージ"})
logger.warning({"user_name": "tanaka", "message": "ワーニングのメッセージ"})
logger.info({"user_name": "tanaka", "message": "インフォのメッセージ"})
logger.debug({"user_name": "tanaka", "message": "デバッグのメッセージ"})

{"asctime": "2024-12-30 00:14:15,512", "levelname": "CRITICAL", "name": "__main__", "funcName": "<module>", "message": "クリティカルのメッセージ", "filename": "2244160616.py", "lineno": 16, "user_name": "tanaka"}
{"asctime": "2024-12-30 00:14:15,513", "levelname": "ERROR", "name": "__main__", "funcName": "<module>", "message": "エラーのメッセージ", "filename": "2244160616.py", "lineno": 17, "user_name": "tanaka"}
{"asctime": "2024-12-30 00:14:15,514", "levelname": "WARNING", "name": "__main__", "funcName": "<module>", "message": "ワーニングのメッセージ", "filename": "2244160616.py", "lineno": 18, "user_name": "tanaka"}
{"asctime": "2024-12-30 00:14:15,515", "levelname": "INFO", "name": "__main__", "funcName": "<module>", "message": "インフォのメッセージ", "filename": "2244160616.py", "lineno": 19, "user_name": "tanaka"}
{"asctime": "2024-12-30 00:14:15,516", "levelname": "DEBUG", "name": "__main__", "funcName": "<module>", "message": "デバッグのメッセージ", "filename": "2244160616.py", "lineno": 20, "user_name": "tanaka"}


## 関数の開始終了時刻を記録するデコレータ

In [3]:
def record_function_start_and_end_time(func):
    def _record_function_start_and_end_time(*args, **kwargs):
        function_name = func.__name__
        logger.debug({"message": f"{function_name} is started."})
        result = func(*args, **kwargs)
        logger.debug({"message": f"{function_name} is completed."})
        return result

    return _record_function_start_and_end_time


@record_function_start_and_end_time
def test_get_double(number: int) -> int:
    return number * 2


print(test_get_double(10))

{"asctime": "2024-12-30 00:15:32,489", "levelname": "DEBUG", "name": "__main__", "funcName": "_record_function_start_and_end_time", "message": "test_get_double is started.", "filename": "3572751676.py", "lineno": 4}
{"asctime": "2024-12-30 00:15:32,491", "levelname": "DEBUG", "name": "__main__", "funcName": "_record_function_start_and_end_time", "message": "test_get_double is completed.", "filename": "3572751676.py", "lineno": 6}


20
